This tutorial demonstrates how to calculate the duration of service station statuses within a day using status log files. These log files record the timestamp of each status change (on/off) for each service station. Our goal is to determine the total time each station was on or off during any given day.

In [0]:
#import modules
import pandas as pd 

from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import datetime 
from pyspark.sql.types import *

The input log file is stored in a Parquet file and contains three columns: station_id, status_change_timestamp, and new_status.

In [0]:
sessions = spark.read.parquet('dbfs:/mnt/test/test_input.parquet' )
sessions.createOrReplaceTempView('status_series') #just for demonstrating some Spark SQL as well
sessions.display()

station_id,status_change_timestamp,new_status
782-1,2024-05-20T12:05:51.000+0000,ON
782-1,2024-04-10T21:52:45.000+0000,ON
778-2,2024-01-28T22:45:00.000+0000,ON
778-2,2024-02-02T13:15:41.000+0000,ON
782-1,2024-02-02T13:15:41.000+0000,ON
778-2,2024-02-02T17:07:04.000+0000,ON
782-1,2024-02-02T17:07:04.000+0000,ON
782-1,2024-05-19T11:22:52.000+0000,ON
778-2,2024-02-22T02:23:09.000+0000,OFF
778-2,2024-05-04T16:02:16.000+0000,OFF


Our result should be something like the below, for a given reference date:

station_id,online_seconds,offline_seconds,Date
778-2,86400,0,2024-05-31
778-1,86400,0,2024-05-31
782-1,86400,0,2024-05-31


The first step is to add the timestamp of the previous update for each service station into the dataset. But before that, it would be best to ensure there are no duplicate entries, for the same station_id and status_change_timestamp.

In [0]:
dups = spark.sql("select station_id,status_change_timestamp from status_series group by station_id,status_change_timestamp having count(*)>1")

dups.count() #if dups has > 0 records there are duplicates


Out[25]: 0

In [0]:
#sessions = spark.sql("select * from status_series") #we can load sessions into a data frame from the underlying view as well

windowSpec =  Window.partitionBy("station_id").orderBy("status_change_timestamp") #create specification for the window function used below

# Add previous and next date_updated for the same connector_id, using PySpark
sessions_laglead = ( sessions
    .withColumn("previous_update", lag("status_change_timestamp").over(windowSpec))
    .withColumn("next_update", lead("status_change_timestamp").over(windowSpec))
)

sessions_laglead.display()


station_id,status_change_timestamp,new_status,previous_update,next_update
778-1,2024-01-28T22:45:00.000+0000,ON,null,2024-02-02T13:15:00.000+0000
778-1,2024-02-02T13:15:00.000+0000,ON,2024-01-28T22:45:00.000+0000,2024-02-02T17:06:16.000+0000
778-1,2024-02-02T17:06:16.000+0000,ON,2024-02-02T13:15:00.000+0000,2024-02-14T04:45:01.000+0000
778-1,2024-02-14T04:45:01.000+0000,OFF,2024-02-02T17:06:16.000+0000,2024-02-22T02:23:10.000+0000
778-1,2024-02-22T02:23:10.000+0000,OFF,2024-02-14T04:45:01.000+0000,2024-05-19T08:41:59.000+0000
778-1,2024-05-19T08:41:59.000+0000,OFF,2024-02-22T02:23:10.000+0000,2024-05-19T09:20:02.000+0000
778-1,2024-05-19T09:20:02.000+0000,OFF,2024-05-19T08:41:59.000+0000,2024-05-19T11:08:29.000+0000
778-1,2024-05-19T11:08:29.000+0000,ON,2024-05-19T09:20:02.000+0000,2024-05-19T11:22:52.000+0000
778-1,2024-05-19T11:22:52.000+0000,ON,2024-05-19T11:08:29.000+0000,2024-06-04T13:45:03.000+0000
778-1,2024-06-04T13:45:03.000+0000,ON,2024-05-19T11:22:52.000+0000,2024-06-04T18:08:48.000+0000


To calculate the online and offline seconds for each station within a given day, we need to determine the time elapsed between subsequent updates. For each station update, there are four cases:

- **Both updates within the same day**: Subtract the timestamp of the current update from the next update to find the duration in seconds.
- **First update of the day**: This is the last update before the day starts. Set the left end of the interval to 12:00:00 AM to calculate the seconds from the start of the day until the first update.
- **Last update of the day**: If the next update is on a subsequent day, constrain the right end of the interval to 12:00:00 AM of the next day. This will give the number of seconds from the last update of the day until the end of the day.
- **No updates within the reference date**: This is an 'extreme' case, in which the above transformations have no meaning (and no result), as the station has not been updated at all, withing the date in reference. In such a case we'll create an 'artificial' record with the status set to the last recorded status of the station, and the duration set to 24 hours (86,400 seconds).


In [0]:
cur_date = datetime.date(2024,5,19) #just an example reference date : 20 May 2024

#convert to text
scur_date_dt= cur_date.strftime("%Y-%m-%d")  # Format as YYYY-MM-DD
sprev_date_dt= (cur_date + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")  # Format prev date as YYYY-MM-DD
snext_date_dt= (cur_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")  # Format next date as YYYY-MM-DD

#get the records for the 1st case (updates within the day)
updates_within_day = sessions_laglead.filter(f"""
                (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                AND (next_update BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                """)
updates_within_day_for_union = updates_within_day.select (
    updates_within_day.station_id,
    updates_within_day.new_status,
    updates_within_day.status_change_timestamp.alias('interval_from'),
    updates_within_day.next_update.alias('interval_to')
)
        
#2nd case: find the latest update of the records BEFORE the current date, then use the time elapsed from 12:00:00 AM to next_update
last_update_of_prev_day = sessions_laglead.filter(f"""
                    (
                    (status_change_timestamp < '{scur_date_dt}')
                        AND (next_update BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                    ) 
                """)
last_update_of_prev_day_for_union = last_update_of_prev_day.select (
    last_update_of_prev_day.station_id,
    last_update_of_prev_day.new_status,
    lit(f"{scur_date_dt} 00:00:00").alias('interval_from'),
    last_update_of_prev_day.next_update.alias('interval_to')
)

#this is a sub-version of case #2, as we could have the 1st update of the day to be the 1st update even for that station
#in such a case we wouldn't have an update for that station before the reference day.
#we would instead have a record with status_change_timestamp within the reference date, and previous_update = NULL, as it would be the first update ever
first_update_ever = sessions_laglead.filter(f"""
                    (
                    (previous_update IS NULL)
                        AND (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                    ) 
                """)
first_update_ever_for_union = first_update_ever.select (
    first_update_ever.station_id,
    first_update_ever.new_status,
    lit(f"{scur_date_dt} 00:00:00").alias('interval_from'),
    first_update_ever.status_change_timestamp.alias('interval_to')
)

#case 3: the last update of the day - the next_update belongs to the next date, while status_change_timestamp is within the date
#a second version here, would be that the last update of the day, would be the last update ever. In such a case next_update = NULL
last_update_of_day = sessions_laglead.filter(f"""
                    (
                    (next_update > '{snext_date_dt}')
                        AND (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                    ) OR
                    (
                        (next_update IS NULL )
                        AND (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                    )

                """)
last_update_of_day_for_union = last_update_of_day.select (
    last_update_of_day.station_id,
    last_update_of_day.new_status,
    last_update_of_day.status_change_timestamp.alias('interval_from'),
    lit(f"{snext_date_dt} 00:00:00").alias('interval_to')
)

#create a data frame with the intervals for all updates within a day               
all_updates_in_day = updates_within_day_for_union.union(first_update_ever_for_union).union(last_update_of_prev_day_for_union).union(last_update_of_day_for_union)

#we also need to take into account the 4th case: stations that have not been updated at all within current date
#find the last update before current date for all stations - first create a suitable specification
windowSpec_desc =  Window.partitionBy("station_id").orderBy(sessions_laglead["status_change_timestamp"].desc())
df_with_row_number = (
    sessions_laglead.filter(f"status_change_timestamp < '{scur_date_dt}' ")
    .withColumn("row_number", row_number().over(windowSpec_desc))  
)
last_records = df_with_row_number.filter(df_with_row_number.row_number == 1).drop("row_number") #we only want to keep the latest record - and also drop the row_number


#find stations missing from current day updates, and keep their last status
missing_stations_in_date = ( 
        last_records.join(  all_updates_in_day, 
        last_records.station_id == all_updates_in_day.station_id,
        "left")
        .filter("interval_from is NULL") #keep only missing stations
        .select(
            last_records.station_id,
            last_records.new_status,
            
        )
        .dropDuplicates() 
)

#create a record with 24h interval for missing stations
missing_stations_in_date_for_union = missing_stations_in_date.select (
    missing_stations_in_date.station_id,
    missing_stations_in_date.new_status,
    lit(f"{scur_date_dt} 00:00:00").alias('interval_from'),
    lit(f"{snext_date_dt} 00:00:00").alias('interval_to')
)

#get them into the dataset to complete the intervals for all stations for current date
all_updates = all_updates_in_day.union(missing_stations_in_date_for_union)

#get online sec and offline sec
online_updates_per_station = (
    all_updates
    .filter("new_status='ON'")
    .withColumn(
        "online_seconds_per_interval",
        unix_timestamp(col("interval_to")) - unix_timestamp(col("interval_from"))
    ).groupBy("station_id")
    .agg (
        sum("online_seconds_per_interval").alias("online_seconds")
    )
) 
offline_updates_per_station = (
    all_updates
    .filter("new_status='OFF'")
    .withColumn(
        "offline_seconds_per_interval",
        unix_timestamp(col("interval_to")) - unix_timestamp(col("interval_from"))
    ).groupBy("station_id")
    .agg (
        sum("offline_seconds_per_interval").alias("offline_seconds")
    )
) 
online_updates_per_station = online_updates_per_station.withColumnRenamed("station_id","online_station_id")
offline_updates_per_station = offline_updates_per_station.withColumnRenamed("station_id","offline_station_id")

#combine the offline and online result into one dataset
availability_per_station = (
    online_updates_per_station
    .join(  offline_updates_per_station, 
            online_updates_per_station.online_station_id == offline_updates_per_station.offline_station_id,
            "outer")
    .select(

        coalesce(online_updates_per_station.online_station_id, offline_updates_per_station.offline_station_id).alias("station_id"),
        online_updates_per_station.online_seconds,
        offline_updates_per_station.offline_seconds
    )
    .fillna(0) #null->0
)

availability_per_station.display()

station_id,online_seconds,offline_seconds
778-1,77610,8790
778-2,77609,8791
782-1,77607,8793


#Do it for many days
We could perhaps create a loop, to perform the previous calculation for an interval of many dates.
We would first need to create a sequence of dates between our start and end dates.

In [0]:
start_date = datetime.date(2024,2,9)
end_date = datetime.date(2024,5,31)

#create a dataframe with all dates from start_date to end_date

date_df = spark.createDataFrame([(start_date, end_date)], ["start", "end"]) \
    .withColumn("start", col("start").cast(DateType())) \
    .withColumn("end", col("end").cast(DateType()))

# Create a range of dates
date_range_df = date_df.select(explode(sequence(col("start"), col("end"))).alias("date"))

# convert to list for the loop
date_range_list = date_range_df.rdd.flatMap(lambda x: x).collect()



In [0]:
#now we canuse the previous code, in a loop
for cur_date in date_range_list:

    #convert to text 
    scur_date_dt= cur_date.strftime("%Y-%m-%d")  # Format as YYYY-MM-DD
    sprev_date_dt= (cur_date + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")  # Format prev date as YYYY-MM-DD
    snext_date_dt= (cur_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")  # Format next date as YYYY-MM-DD

    #get the records for the 1st case (updates within the day)
    updates_within_day = sessions_laglead.filter(f"""
                    (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                    AND (next_update BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                    """)
    updates_within_day_for_union = updates_within_day.select (
        updates_within_day.station_id,
        updates_within_day.new_status,
        updates_within_day.status_change_timestamp.alias('interval_from'),
        updates_within_day.next_update.alias('interval_to')
    )
            
    #2nd case: find the latest update of the records BEFORE the current date, then use the time elapsed from 12:00:00 AM to next_update
    last_update_of_prev_day = sessions_laglead.filter(f"""
                        (
                        (status_change_timestamp < '{scur_date_dt}')
                            AND (next_update BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                        ) 
                    """)
    last_update_of_prev_day_for_union = last_update_of_prev_day.select (
        last_update_of_prev_day.station_id,
        last_update_of_prev_day.new_status,
        lit(f"{scur_date_dt} 00:00:00").alias('interval_from'),
        last_update_of_prev_day.next_update.alias('interval_to')
    )

    #this is a sub-version of case #2, as we could have the 1st update of the day to be the 1st update even for that station
    #in such a case we wouldn't have an update for that station before the reference day.
    #we would instead have a record with status_change_timestamp within the reference date, and previous_update = NULL, as it would be the first update ever
    first_update_ever = sessions_laglead.filter(f"""
                        (
                        (previous_update IS NULL)
                            AND (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                        ) 
                    """)
    first_update_ever_for_union = first_update_ever.select (
        first_update_ever.station_id,
        first_update_ever.new_status,
        lit(f"{scur_date_dt} 00:00:00").alias('interval_from'),
        first_update_ever.status_change_timestamp.alias('interval_to')
    )

    #case 3: the last update of the day - the next_update belongs to the next date, while status_change_timestamp is within the date
    #a second version here, would be that the last update of the day, would be the last update ever. In such a case next_update = NULL
    last_update_of_day = sessions_laglead.filter(f"""
                        (
                        (next_update > '{snext_date_dt}')
                            AND (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                        ) OR
                        (
                            (next_update IS NULL )
                            AND (status_change_timestamp BETWEEN '{scur_date_dt}' AND '{snext_date_dt}')
                        )

                    """)
    last_update_of_day_for_union = last_update_of_day.select (
        last_update_of_day.station_id,
        last_update_of_day.new_status,
        last_update_of_day.status_change_timestamp.alias('interval_from'),
        lit(f"{snext_date_dt} 00:00:00").alias('interval_to')
    )

    #create a data frame with the intervals for all updates within a day               
    all_updates_in_day = updates_within_day_for_union.union(first_update_ever_for_union).union(last_update_of_prev_day_for_union).union(last_update_of_day_for_union)

    #we also need to take into account the 4th case: stations that have not been updated at all within current date
    #find the last update before current date for all stations - first create a suitable specification
    windowSpec_desc =  Window.partitionBy("station_id").orderBy(sessions_laglead["status_change_timestamp"].desc())
    df_with_row_number = (
        sessions_laglead.filter(f"status_change_timestamp < '{scur_date_dt}' ")
        .withColumn("row_number", row_number().over(windowSpec_desc))  
    )
    last_records = df_with_row_number.filter(df_with_row_number.row_number == 1).drop("row_number") #we only want to keep the latest record - and also drop the row_number


    #find stations missing from current day updates, and keep their last status
    missing_stations_in_date = ( 
            last_records.join(  all_updates_in_day, 
            last_records.station_id == all_updates_in_day.station_id,
            "left")
            .filter("interval_from is NULL") #keep only missing stations
            .select(
                last_records.station_id,
                last_records.new_status,
                
            )
            .dropDuplicates() 
    )

    #create a record with 24h interval for missing stations
    missing_stations_in_date_for_union = missing_stations_in_date.select (
        missing_stations_in_date.station_id,
        missing_stations_in_date.new_status,
        lit(f"{scur_date_dt} 00:00:00").alias('interval_from'),
        lit(f"{snext_date_dt} 00:00:00").alias('interval_to')
    )

    #get them into the dataset to complete the intervals for all stations for current date
    all_updates = all_updates_in_day.union(missing_stations_in_date_for_union)

    #get online sec and offline sec
    online_updates_per_station = (
        all_updates
        .filter("new_status='ON'")
        .withColumn(
            "online_seconds_per_interval",
            unix_timestamp(col("interval_to")) - unix_timestamp(col("interval_from"))
        ).groupBy("station_id")
        .agg (
            sum("online_seconds_per_interval").alias("online_seconds")
        )
    ) 
    offline_updates_per_station = (
        all_updates
        .filter("new_status='OFF'")
        .withColumn(
            "offline_seconds_per_interval",
            unix_timestamp(col("interval_to")) - unix_timestamp(col("interval_from"))
        ).groupBy("station_id")
        .agg (
            sum("offline_seconds_per_interval").alias("offline_seconds")
        )
    ) 
    online_updates_per_station = online_updates_per_station.withColumnRenamed("station_id","online_station_id")
    offline_updates_per_station = offline_updates_per_station.withColumnRenamed("station_id","offline_station_id")

    #combine the offline and online result into one dataset
    availability_per_station = (
        online_updates_per_station
        .join(  offline_updates_per_station, 
                online_updates_per_station.online_station_id == offline_updates_per_station.offline_station_id,
                "outer")
        .select(

            coalesce(online_updates_per_station.online_station_id, offline_updates_per_station.offline_station_id).alias("station_id"),
            online_updates_per_station.online_seconds,
            offline_updates_per_station.offline_seconds
        )
        .fillna(0) #null->0
    )

    #add the date into the dataframe
    availability_per_station = availability_per_station.withColumn("Date",lit(cur_date))
    
    #initialize the all_dates dataframe in the first iteration, append to it outherwise
    if cur_date == start_date:
        availability_per_station_all_dates = availability_per_station
    else:
        availability_per_station_all_dates = availability_per_station_all_dates.union(availability_per_station) 

#end of loop

availability_per_station_all_dates.display()


station_id,online_seconds,offline_seconds,Date
778-2,86400,0,2024-02-09
778-1,86400,0,2024-02-09
782-1,86400,0,2024-02-09
778-1,86400,0,2024-02-10
778-2,86400,0,2024-02-10
782-1,55201,31199,2024-02-10
778-1,86400,0,2024-02-11
778-2,86400,0,2024-02-11
782-1,0,86400,2024-02-11
778-1,86400,0,2024-02-12


As you can see, if a station has not yet received an update, it is absent from the result set. This is because our code assumes that a station will receive its first update (ON or OFF) upon installation, making it usable. This assumption fits into a process where new stations are installed over time.

#Ideas for next steps

In a production data flow (i.e., notebook), it's often preferable to minimize the use of loops by 'vectorizing' calculations, as we would say in R programming. So instead of having a a loop, we could create the "updates_within_day_for_union", "last_update_of_prev_day_for_union", etc. data frames for all dates / stations and then perform the necessary aggregations in the unioned data frame. I believe the 'loop' approach is better, from a learning perspective, as it's more straightforward.

We could also perform **incremental loading**: we could store the last processed date in a Delta Table, starting with a date in the past. The notebook would read the start_date from this table (actually start_date -1), process data up to yesterday, and then store the last processed date back into the table for the next execution.

I won't cover these advanced techniques in this tutorial. I might create another notebook in the future to demonstrate these approaches.

I hope you find the tips shared here helpful in your work :-)